In [73]:
#In this example, you'll be creating a Python script to visualize the weather of 500+ cities across the world 
#of varying distance from the equator. To accomplish this, you'll be utilizing a simple Python library,
#the OpenWeatherMap API, and a little common sense to create a representative model of weather across world cities.

#Your objective is to build a series of scatter plots to showcase the following relationships:

# Temperature (F) vs. Latitude
# Humidity (%) vs. Latitude
# Cloudiness (%) vs. Latitude
# Wind Speed (mph) vs. Latitude

#Note : Coording to AcrcMap( lonk is given in READ ME):
#        1) (lat) Latitude_range = -90 to + 90
#        2)  (lon) Longitude_range = -180 to +180

#  Referance GP_cities.py to get lat, lon, cities combination data

In [74]:
#Dependancies
import pandas as pd
import matplotlib.pyplot as plt
from citipy import citipy as cp

In [75]:
# We now find the combination of  ( lat,lon) by creating loop for lat in the range -90 to +90, and, lon in the range -180 to 180.
# The we will find the city name corrosponding to that (lat,lon) using citipy module 
# There will be thousands of combunations of (lat,lon). So, we will give by skipping it by 5 in the range.
citylist = []
count = 0
dup = 'no'

for lat in range(-90, 90, 5):
    for lon in range(-180, 180, 10):
        cp_city = cp.nearest_city(lat,lon) # Capture nearest city from Citipi library
        
        # Use a dictionary to store info of the current city:
        citydict = {} 
        citydict['City_Name'] = cp_city.city_name 
        citydict['Country']= cp_city.country_code
        citydict['Latitude'] = lat
        citydict['Longitude'] = lon
        
        # Add current city info to list: <citylist>:
        if len(citylist) == 0:
            citylist.append(citydict)
            count += 1
            continue
        else:
            #Get rid of duplicates:
            for city_info in citylist:
                if city_info['City_Name'] == citydict['City_Name']:
                    dup = 'yes'
        if dup == 'no':
            citylist.append(citydict)
            count += 1
        else:
            dup = 'no'

print(len(citylist))

588


In [76]:
import json
import requests as req

In [77]:
# create dataframe to 
citypd = pd.DataFrame({
    "City": [x["City_Name"] for x in citylist],
    "Country": [x["Country"] for x in citylist]
})

citypd.head()

,City,Country
0,vaini,to
1,mataura,pf
2,rikitea,pf
3,punta arenas,cl
4,ushuaia,ar


In [87]:
# import config.py file to read OpenWeatherMap API key, and hide it from the code:
import config

url = "https://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"
base_url = url + "appid=" + config.weather_api_key + "&units=" + units + "&q=" 

In [83]:
count = 0

cities_init = {
    'City': "", 
    'Country': "", 
    'Lat': "", 
    'Lon': "", 
    'Temp': "", 
    'Humidity': "", 
    'Cloudiness': "", 
    'Wind Speed': ""
}
cities500 = pd.DataFrame(data=cities_init, index=[0])

for index,row in citypd.iterrows():
    query_url = base_url + row['City'] + "," + row['Country']
    try:
        weather_response = req.get(query_url)
        cityweather = weather_response.json()

        cities500.set_value(count, "City", cityweather['name'])
        cities500.set_value(count, "Country", cityweather['sys']['country'])
        cities500.set_value(count, "Lat", int(cityweather['coord']['lat']))
        cities500.set_value(count, "Lon", int(cityweather['coord']['lon']))
        cities500.set_value(count, "Temp", int(cityweather['main']['temp']))
        cities500.set_value(count, "Humidity", int(cityweather['main']['humidity']))
        cities500.set_value(count, "Cloudiness", int(cityweather['clouds']['all']))
        cities500.set_value(count, "Wind Speed", int(cityweather['wind']['speed']))
        count+= 1
    except:
        print(f"No data for this city: {row['City'],row['Country']}")

cities500 = cities500[['City', 'Country', 'Cloudiness', 'Humidity', 'Lat', 'Lon', 'Temp', 'Wind Speed']]
cities500.head()

No data for this city: ('mataura', 'pf')
No data for this city: ('taolanaro', 'mg')
No data for this city: ('tsihombe', 'mg')
No data for this city: ('laguna', 'br')
No data for this city: ('umzimvubu', 'za')
No data for this city: ('marcona', 'pe')
No data for this city: ('toliary', 'mg')
No data for this city: ('grand river south east', 'mu')
No data for this city: ('bengkulu', 'id')
No data for this city: ('halalo', 'wf')
No data for this city: ('satitoa', 'ws')
No data for this city: ('mocambique', 'mz')
No data for this city: ('palabuhanratu', 'id')
No data for this city: ('vaitupu', 'wf')
No data for this city: ('samalaeulu', 'ws')
No data for this city: ('miranorte', 'br')
No data for this city: ('nguiu', 'au')
No data for this city: ('alotau', 'pg')
No data for this city: ('lata', 'sb')
No data for this city: ('saleaula', 'ws')
No data for this city: ('utiroa', 'ki')
No data for this city: ('samusu', 'ws')
No data for this city: ('tucuma', 'br')
No data for this city: ('acarau'

,City,Country,Cloudiness,Humidity,Lat,Lon,Temp,Wind Speed
0,Vaini,TO,90,94,-21,-175,78,10
1,Rikitea,PF,92,100,-23,-134,78,10
2,Punta Arenas,CL,75,62,-53,-70,55,17
3,Ushuaia,AR,40,50,-54,-68,55,18
4,Hermanus,ZA,0,75,-34,19,53,3


In [84]:
len(cities500.index)

503